# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import json
import azureml.core
import joblib
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

print("SDK version:", azureml.core.VERSION)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')



In [ ]:
experiment_name = 'hyper-exp'
experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

In [ ]:

aml_compute_cluster = "heartFailure-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=aml_compute_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, aml_compute_cluster, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
print(compute_target.get_status().serialize())

In [ ]:
key = "HeartFailure-Dataset"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1,  delay_evaluation = 3)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
        "--C": uniform(0.1,1.0),
        "--max_iter": choice(50, 100, 150, 200)
    })

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='./', 
                    entry_script='train.py',
                    compute_target=compute_target)

hyperdrive_run_config = HyperDriveConfig(estimator = estimator,
                                hyperparameter_sampling = param_sampling,
                                policy = early_termination_policy,
                                primary_metric_name = 'Accuracy',
                                primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs = 20,
                                max_concurrent_runs = 4)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()


In [ ]:
hyperdrive_run.get_status()


In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_hyper_run = hyper_run.get_best_run_by_primary_metric()
best_hyper_run_metrics = best_hyper_run.get_metrics()
parameter_values = best_hyper_run.get_details() ['runDefinition']['arguments']

os.makedirs("./outputs", exist_ok=True)
joblib.dump(value=best_hyper_run.id,filename='outputs/best_hyper_run_model.joblib')


print('Best Run Id: ', best_hyper_run.id)
print('\n Accuracy: ', best_hyper_run_metrics['Accuracy'])
print('\n Metrics: ', best_hyper_run_metrics)
print('\n Parameters: ', parameter_values)

In [ ]:
best_run

In [ ]:
#TODO: Save the best model
best_hd_model = best_hyper_run.register_model(model_name='best_hyperdrive_model', model_path='outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service